In [0]:
import pandas as pd

df_indicium = pd.read_csv("filmes_tratado.csv")

In [0]:
import numpy as np

# Criar coluna primary_genre (primeiro gênero listado)
df_indicium['primary_genre'] = df_indicium['Genre'].astype(str).str.split(',').str[0].str.strip()

# Criar logs (No_of_Votes/Gross)
df_indicium['log_No_of_Votes'] = np.log1p(df_indicium['No_of_Votes'])
df_indicium['log_Gross'] = np.log1p(df_indicium['Gross'])

# Verificações rápidas
print("Shape:", df_indicium.shape)
print(df_indicium[['IMDB_Rating','Meta_score','Runtime','No_of_Votes','Gross']].describe().T)

Shape: (999, 18)
             count          mean  ...         75%          max
IMDB_Rating  999.0  7.947948e+00  ...         8.1          9.2
Meta_score   999.0  7.796912e+01  ...        85.5        100.0
Runtime      999.0  1.228719e+02  ...       137.0        321.0
No_of_Votes  999.0  2.716214e+05  ...    373167.5    2303232.0
Gross        999.0  6.053338e+07  ...  61576564.5  936662225.0

[5 rows x 8 columns]


In [0]:
# Pacote para XGBoost
%pip install xgboost 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 60.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Pacote para NLTK
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.4/797.4 kB 12.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Downloads necessários do nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('punkt_tab')

# Função de pré-processamento do texto
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    if pd.isnull(text):
        return ""
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t.isalpha()]  # remove pontuação/números
    tokens = [t for t in tokens if t not in stop_words]  # remove stopwords
    return " ".join(tokens)

# Aplica o pré-processamento ao texto
df_indicium["Overview"] = df_indicium["Overview"].apply(preprocess_text)
df_indicium["Series_Title"] = df_indicium["Series_Title"].apply(preprocess_text)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xff08bc91dd00>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'
[n

In [0]:
# Separa features e target
X = df_indicium.drop(columns=["IMDB_Rating"])
y = df_indicium["IMDB_Rating"]

# Features numéricas e categóricas
numeric_features = ['Released_Year', 'Runtime', 'Meta_score', 'log_No_of_Votes', 'log_Gross']
categorical_features = ['Certificate', 'primary_genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']
text_features = ["Overview", "Series_Title"]

In [0]:
# Transformers
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

text_transformer = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(max_features=5000))
])

# Column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("text1", text_transformer, "Overview"),
        ("text2", text_transformer, "Series_Title")
    ]
)

# Dicionário de pipelines para cada modelo
models = {
    "RandomForest": Pipeline([
        ("preprocessor", preprocessor),
        ("model", RandomForestRegressor(random_state=42))
    ]),
    "XGBoost": Pipeline([
        ("preprocessor", preprocessor),
        ("model", XGBRegressor(random_state=42, objective="reg:squarederror", n_jobs=-1))
    ])
}


In [0]:
# Separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [0]:
# Dicionário de grades de hiperparâmetros
param_grids = {
    "RandomForest": {
        "model__n_estimators": [100, 200, 500],
        "model__max_depth": [5, 10, None]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200, 500],
        "model__max_depth": [3, 5, 7],
        "model__learning_rate": [0.05, 0.1]
    }
}



# loop para treinar ambos os modelos
results = []
for name, pipeline in models.items():
    print(f"\nRodando GridSearch para {name}...")
    grid = GridSearchCV(pipeline, param_grids[name], cv=3, scoring="neg_mean_absolute_error")
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"Melhores hiperparâmetros ({name}): {grid.best_params_}")
    print(f"MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")
    
    results.append({
        "Modelo": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "Melhores Hiperparâmetros": grid.best_params_
    })

# Tabela comparativa
results_df = pd.DataFrame(results)
print("\nComparativo final:")
print(results_df)



Rodando GridSearch para RandomForest...
Melhores hiperparâmetros (RandomForest): {'model__max_depth': None, 'model__n_estimators': 500}
MAE: 0.1555 | RMSE: 0.2028 | R²: 0.3735

Rodando GridSearch para XGBoost...
Melhores hiperparâmetros (XGBoost): {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 100}
MAE: 0.1617 | RMSE: 0.2070 | R²: 0.3472

Comparativo final:
         Modelo  ...                           Melhores Hiperparâmetros
0  RandomForest  ...  {'model__max_depth': None, 'model__n_estimator...
1       XGBoost  ...  {'model__learning_rate': 0.1, 'model__max_dept...

[2 rows x 5 columns]


In [0]:
display(results_df)

Modelo,MAE,RMSE,R2,Melhores Hiperparâmetros
RandomForest,0.15547099999999925,0.20279971942781508,0.3734731856681418,"List(null, null, 500)"
XGBoost,0.1617183117866516,0.20700512243660396,0.3472195369722537,"List(0.1, 5, 100)"


In [0]:
# Dados para previsão
teste = {
    'Series_Title': 'The Shawshank Redemption',
    'Released_Year': 1994,
    'Certificate': 'A',
    'Runtime': '142 min',
    'Genre': 'Drama',
    'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': '28,341,469'
}

# Transformar em DataFrame
df_teste = pd.DataFrame([teste])

# Ajustar campos numéricos
df_teste['Released_Year'] = df_teste['Released_Year'].astype(int)
df_teste['Runtime'] = df_teste['Runtime'].str.replace(' min', '').astype(float)
df_teste['Gross'] = df_teste['Gross'].str.replace(',', '').astype(float)

# Criar colunas derivadas
df_teste['log_No_of_Votes'] = np.log1p(df_teste['No_of_Votes'])
df_teste['log_Gross'] = np.log1p(df_teste['Gross'])

# Criar coluna primary_genre (igual ao treino)
df_teste['primary_genre'] = df_teste['Genre'].str.split(',').str[0]

# Aplicar o mesmo pré-processamento de texto
df_teste["Overview"] = df_teste["Overview"].apply(preprocess_text)

# Manter as colunas do treino
df_teste = df_teste[X.columns]

# Usar o melhor modelo RandomForest já treinado
best_rf = grid  # ou grid.best_estimator_ se for o objeto GridSearch
pred = best_rf.predict(df_teste)[0]

print(f"🎬 Nota prevista no IMDB para {teste['Series_Title']}: {pred:.2f}")


🎬 Nota prevista no IMDB para The Shawshank Redemption: 8.68


In [0]:
import joblib

# salvar o melhor modelo RandomForest treinado
joblib.dump(grid.best_estimator_, "modelo_randomforest.pkl")

print("✅ Modelo salvo como modelo_randomforest.pkl")

# exemplo de como carregar depois
modelo_carregado = joblib.load("modelo_randomforest.pkl")

# testar previsão novamente
pred = modelo_carregado.predict(df_teste)[0]
print(f"🎬 Previsão carregada do modelo salvo: {pred:.2f}")


✅ Modelo salvo como modelo_randomforest.pkl
🎬 Previsão carregada do modelo salvo: 8.68
